In [1]:
# Look The Note Below!!!

import numpy as np
import time

N = 16 # You shoule change the test size here !!!

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# Read the note First ! ! !
#### In all the code above, only two of the function usages are needed. One is is_valid(self, state), which is to determine if the current state is legal; the other is is_satisfy(self, state), which is to determine if the current board meets the win condition. 
#### The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
#### In the first line of the code you can see N = 5, this is the size of the test.
#### Then in the test_block that follows, render indicates whether to use a graphical interface representation, and then method indicates which bts are used(bts or improving_bts).

### Request 1: You should complete the function bts(problem). 
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. (**DDL: 10.28**)
### Request 2: You should complete the function improving_bts(problem). 
You can select one or more methods of the three methods below(Minimum Remaining Value, Least constraining value, Forward checking), but you should have a good performance **when N = 16 without GUI**. (**DDL: 11.04**)

# BTS: Backtracking search (Request 1, DDL: 10.28)

In [2]:
def bts(problem):
    import random
    action_stack = []
    col = np.zeros(N, dtype=int)
    lob = np.zeros(2*N-1, dtype=int)
    rob = np.zeros(2*N-1, dtype=int)
    for i in range(N):
        c = random.randint(0, N-1)
        action_stack.append((i, c))
        col[c] += 1
        lob[i+c] += 1
        rob[N-1-i+c] += 1
    while not problem.is_satisfy(action_stack):
        for i in range(0, N):
            pos = action_stack[i]
            r = pos[0]; c = pos[1]
            col[c] -= 1
            lob[r+c] -= 1
            rob[N-1-r+c] -= 1
            id = -1
            minp = 3*N
            for j in range(0, N):
                p = col[j]+lob[r+j]+rob[N-1-r+j]
                if minp >= p:
                    minp = p
                    if id == -1 or minp > p:
                        id = j
                    elif minp == p and random.randint(0, 1) == 1:
                        id = j
            action_stack[i] = (r, id)
            col[id] += 1
            lob[r+id] += 1
            rob[N-1-r+id] += 1
        yield action_stack
        # print(col)
        # print(lob)
        # print(rob)


# Improving BTS To DO (Request 2, DDL: 11.04)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [3]:
def improving_bts(problem):
    action_stack = []
    id = 0
    col = np.zeros(N+1, dtype=int)
    lob = np.zeros(N+1, dtype=int)
    rob = np.zeros(N+1, dtype=int)
    while not problem.is_satisfy(action_stack):
        flag = False; flag2 = True
        r = len(action_stack)
        if id == 0 and r >= 8:
            for i in my_range(r, N-1):
                t = (col[i] | lob[i] | rob[i])
                col[i+1] = col[i]
                lob[i+1] = lob[i] >> 1
                rob[i+1] = rob[i] << 1
                if (t & ((1<<N)-1)) == (1<<N)-1:
                    flag2 = False
                    break
        # print(flag2)
        if id <= N-1 and flag2 == True:
            t = (col[r] | lob[r] | rob[r])
            for i in my_range(id,N-1):
                p = 1<<i
                if (t & p) == 0:
                    action_stack.append((r, i))
                    flag = True
                    col[r+1] = col[r] | p
                    lob[r+1] = (lob[r] | p) >> 1
                    rob[r+1] = (rob[r] | p) << 1
                    yield action_stack
                    break
        if flag == False:
            pos = action_stack.pop()
            id = pos[1]+1
        else:
            id = 0
            


In [4]:

# test_block
n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
render = False # here to select GUI or not
method = improving_bts  # here to change your method; bts or improving_bts
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in method(p):
        # p.print_state(actions)
        pass
    p.print_state(actions)
    print(time.time() - start_time)


_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
---------------------------------
4.104996919631958
